In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Establish a connection to the Sakila database
def create_db_engine(user, password, host, port, database):
    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)
    return engine

# Function to retrieve rental data for a given month and year
def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, customer_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(query, engine)
    return df

# Function to count rentals per customer for a given month and year
def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

# Function to compare rentals between two months
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    combined_df['difference'] = combined_df.iloc[:, 1] - combined_df.iloc[:, 2]
    return combined_df

# Example usage
if __name__ == "__main__":
    engine = create_db_engine('user', 'password', 'localhost', '3306', 'sakila')
    
    # Retrieve rental data for May and June
    may_rentals = rentals_month(engine, 5, 2005)
    june_rentals = rentals_month(engine, 6, 2005)
    
    # Count rentals per customer for May and June
    may_rental_count = rental_count_month(may_rentals, 5, 2005)
    june_rental_count = rental_count_month(june_rentals, 6, 2005)
    
    # Compare rentals between May and June
    comparison_df = compare_rentals(may_rental_count, june_rental_count)
    print(comparison_df)